In [71]:
from generation.ontology.event_retriever import EventRetriever
from generation.ontology.similarity_calculator import LocalSemanticSimilarityCalculator
from generation.adaptation.astar import ConstructiveAdaptation
from generation.adaptation.query import Query
from common.utils.loader import load_json_folder, load_txt_folder, data_dir, out_dir, save_raw_folktale, load_json
from generation.ontology.folktale_graph import FolktaleOntology
from loguru import logger
from rdflib import Graph
from dotenv import load_dotenv
from generation.story_generator import generate_story
from common.models.folktale import AnnotatedFolktale
import os


In [72]:
def create_graph(build: bool=False, render_html: bool=False) -> Graph:
	graph = FolktaleOntology()
	if build:
		hierarchies = load_json_folder(f"{data_dir}/hierarchies")
		graph.build(hierarchies)

		examples = load_json_folder(f"{data_dir}/examples/annotated")
		for folktale in examples.values():
			graph.add_folktale(folktale)

		graph.add_imports()
		graph.save()
		if render_html:
			graph.render_html("instances")

	logger.debug(f"Grafo initialized with {len(graph)} triplets.")

	graph.load()

	logger.debug(f"Grafo initialized with {len(graph)} triplets after loading.")

	return graph

In [73]:
graph = create_graph(build=False,
						 render_html=False)
	
event_retriever = EventRetriever(graph)
sim_calculator = LocalSemanticSimilarityCalculator(graph)

2026-01-07 00:45:23.088 | DEBUG    | __main__:create_graph:16 - Grafo initialized with 0 triplets.
2026-01-07 00:45:23.126 | DEBUG    | __main__:create_graph:20 - Grafo initialized with 1112 triplets after loading.


In [74]:
def execute_query(query: str):
	try:
		results = graph.query(query)
		result_list = list(results)
		return result_list
	except Exception as e:
		print("Error in SPARQL query:", e)
		return []

In [75]:
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS

ONT = Namespace("https://rbc.org/ontology/")
RES = Namespace("https://rbc.org/resources/")

WD = Namespace("http://www.wikidata.org/entity/")		# elementos
WDT = Namespace("http://www.wikidata.org/prop/direct/")	# propiedades
SCHEMA = Namespace("https://schema.org/")

SEM = Namespace("http://semanticweb.cs.vu.nl/2009/11/sem/")
REL = Namespace("http://purl.org/vocab/relationship/")
PEARL = Namespace("https://www.gsi.upm.es/ontologies/pearl/")


In [76]:
def get_folktales_by_genre(genre: str) -> list[str]:
	genre_map = {
		"fable": "wd:Q693",
		"fairy_tale": "wd:Q699",
		"legend": "wd:Q44342",
		"myth": "wd:Q12827256",
		"tall_tale": "wd:Q1951220",
	}

	if genre not in genre_map:
		raise ValueError(f"Unsupported genre: {genre}")

	genre_wd = genre_map[genre]

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>

	SELECT ?folktale ?title
	WHERE {{
		?folktale a ont:Folktale ;
		   ont:hasGenre {genre_wd} ;
		   ont:title ?title .
	}}
	"""
	results = execute_query(query)

	return [(str(result.folktale), str(result.title))for result in results] if results else []

In [77]:
get_folktales_by_genre("fable")

[('https://rbc.org/resources/folktale/the_three_little_pigs',
  'The Three Little Pigs'),
 ('https://rbc.org/resources/folktale/the_hare_and_the_tortoise',
  'The Hare and The Tortoise')]

In [78]:
def get_roles_by_type_and_folktale_genre(role_type,genre
) -> list[tuple[str, str, str]]:
	"""
	Returns:
		[(role_uri, role_label, folktale_title), ...]
	"""

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX rdfs: <{RDFS}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>


	SELECT DISTINCT ?agent ?roleLabel ?folktaleTitle
	WHERE {{
		?folktale a ont:Folktale ;
				  ont:hasGenre <{genre}> ;
				  ont:title ?folktaleTitle ;
				  ont:hasEvent ?event .

		?event ont:hasAgent ?agent .
		?agent ont:hasRole ?role .

		?role rdf:type <{role_type}> .
		OPTIONAL {{ ?role rdfs:label ?roleLabel }}
	}}
	"""

	results = execute_query(query)

	return [
		(str(row.agent), str(row.roleLabel), str(row.folktaleTitle))
		for row in results
	]


In [79]:
roles = get_roles_by_type_and_folktale_genre(
	role_type=ONT.Hero,
	genre=FolktaleOntology.GENRE_MAP["fable"]
)


In [80]:
roles

[('https://rbc.org/resources/agent/the_three_little_pigs/pig_three',
  'Hero',
  'The Three Little Pigs'),
 ('https://rbc.org/resources/agent/the_hare_and_the_tortoise/tortoise',
  'Hero',
  'The Hare and The Tortoise')]

In [81]:
def get_ordered_events_for_agent(agent_uri) -> list[tuple[str, str]]:
    query = f"""
    PREFIX rdfs: <{RDFS}>
    PREFIX ont: <{ONT}>

    SELECT ?event ?label ?eventType (COUNT(?prev) AS ?order)
    WHERE {{
        ?event ont:hasAgent <{agent_uri}> ;
            rdf:type ?eventType .
        OPTIONAL {{ ?event rdfs:label ?label }}

        OPTIONAL {{
            ?prev ont:postEvent+ ?event .
        }}
    }}
    GROUP BY ?event ?label
    ORDER BY ?order
    """
    results = execute_query(query)

    return [
        (str(r.eventType),str(r.event), str(r.label),str(r.order))
        for r in results
    ]


In [82]:
agent_uri = "https://rbc.org/resources/role/the_three_little_pigs/hero"

events = get_ordered_events_for_agent(roles[0][0])

for eventType,event_uri, label,order in events:
    print(eventType)

https://rbc.org/ontology/InitialSituation
https://rbc.org/ontology/VillainDefeated


In [83]:
events

[('https://rbc.org/ontology/InitialSituation',
  'https://rbc.org/resources/event/the_three_little_pigs/build_sturdy_house',
  'Build Sturdy House',
  '2'),
 ('https://rbc.org/ontology/VillainDefeated',
  'https://rbc.org/resources/event/the_three_little_pigs/villain_fails_sturdy_house',
  'Villain Fails Sturdy House',
  '5')]

In [ ]:
def get_role_classes(event_uri: str):
		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT ?roleClass (COUNT(?role) AS ?roleCount) (GROUP_CONCAT(DISTINCT ?agent; separator=", ") AS ?agents)
		WHERE {{
			<{event_uri}> ont:hasAgent ?agent .
			?agent ont:hasRole ?role .
			?role rdf:type ?roleClass .
		}}
		GROUP BY ?roleClass
		"""

		results = execute_query(query)

		if results:
			dict_result = {str(result.roleClass).split('/')[-1]: (int(result.roleCount), str(result.agents).split(',')) for result in results}
			return dict_result
		return {}


In [ ]:
def get_object_classes(event_uri: str):
    query = f"""
    PREFIX rdfs: <{RDFS}>
    PREFIX rdf: <{RDF}>
    PREFIX ont: <{ONT}>

    SELECT ?objectClass (COUNT(?object) AS ?objectCount) (GROUP_CONCAT(DISTINCT ?object; separator=",") AS ?objects)
    WHERE {{
        <{event_uri}> ont:hasObject ?object .
        ?object rdf:type ?objectClass .
    }}
    GROUP BY ?objectClass
    """

    results = execute_query(query)

    if results:
        dict_result = {str(result.objectClass).split('/')[-1]: (int(result.objectCount),str(result.objects).split(',') ) for result in results}
        return dict_result

    return {}


In [126]:
get_object_classes("https://rbc.org/resources/event/the_three_little_pigs/villain_fails_sturdy_house")

{}

In [125]:
d["HeroFamily"][1][0]

'https://rbc.org/resources/agent/the_three_little_pigs/pig_two'